# Binance Test Net API Research

In [ ]:
import pandas as pd
import numpy as np
from binance.client import Client
import datetime

import requests
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import websocket
import json

import plotly.graph_objs as go

: 

In [ ]:
### Testnet URLs
# https://testnet.binance.vision/api
# wss://testnet.binance.vision/ws-api/v3
# wss://testnet.binance.vision/ws
# wss://testnet.binance.vision/stream

# Binance Spot Test Network: https://testnet.binance.vision/
# API Key: odwF9bVsSsxjZnckgbSu3NfUgGqqJ2sow4OelwjEttIBB08r3Z5umQL0A03lp2Gd
# Secret key: Q3bcPKvbvlVpzv5BQe3lj7EkWdRhevEp24Oi7TENce6xO0FiXUNQKDa47QTyyKcK

In [33]:
# Global variables for storing price data
timestamps = []
price = {'open':[], 'high':[], 'low':[], 'close':[]}
max_messages = 5  # Maximum number of messages to receive before closing websocket

# Initial dashboard graph
fig = go.Figure()

def time_conv(timestamp):
    return datetime.datetime.fromtimestamp(timestamp/ 1000).strftime('%Y-%m-%d %H:%M:%S.%f')[:-4]

def on_error(ws, error):
    print("WebSocket error:", error)

def on_open(ws):
    print("WebSocket connection opened")

def on_close(ws, message):
    fig.show()
    print("WebSocket connection closed")

def on_message(ws, message):
    global data
    data = json.loads(message)
    # Extract timestamp and price from the data
    timestamp = data['E']

    # Append timestamp and price to the lists
    timestamps.append(time_conv(data['E']))
    price['open'].append(data['o'])
    price['high'].append(data['h'])
    price['low'].append(data['l'])
    price['close'].append(data['c'])

    # Update dashboard graph with new data
    update_graph()

    # Terminate the WebSocket connection after reaching the maximum number of messages
    if len(timestamps) >= max_messages:
        ws.close()
    
'''# Define the callback to update the graph
@app.callback(Output('live-graph', 'figure'),
              events=[Event('interval', 'interval')])'''
def update_graph():
    # Update the graph
    fig.update_xaxes(range=[min(timestamps), max(timestamps)])
    fig.update_yaxes(range=[min(price['open']), max(price['open'])])
    fig.update_traces(
        x=timestamps,
        open=price['open'],
        high=price['high'],
        low=price['low'],
        close=price['close']
    )
    #return fig

# Binance WebSocket API endpoint for BTCUSDT symbol
ws_url = 'wss://testnet.binance.vision/ws/btcusdt@miniTicker'

# Start WebSocket connection
ws = websocket.WebSocketApp(
        url=ws_url,
        on_open=on_open,
        on_message=on_message,
        on_error=on_error,
        on_close=on_close
    )
ws.run_forever()

WebSocket connection opened


False

In [34]:
ws.keep_running

False

In [35]:
ws.close()

In [4]:
data

{'e': 'kline',
 'E': 1685512488852,
 's': 'BTCUSDT',
 'k': {'t': 1685512440000,
  'T': 1685512499999,
  's': 'BTCUSDT',
  'i': '1m',
  'f': 3860977,
  'L': 3861086,
  'o': '27070.14000000',
  'c': '27081.51000000',
  'h': '27081.51000000',
  'l': '27064.40000000',
  'v': '3.52532900',
  'n': 110,
  'x': False,
  'q': '95428.95098110',
  'V': '3.24225800',
  'Q': '87767.02404531',
  'B': '0'}}

In [37]:
timestamps

['2023-05-31 19:03:15.62',
 '2023-05-31 19:03:16.25',
 '2023-05-31 19:03:26.22',
 '2023-05-31 19:03:28.81',
 '2023-05-31 19:03:30.31']

In [39]:
price

{'open': ['27754.31000000',
  '27754.31000000',
  '27754.31000000',
  '27754.31000000',
  '27754.30000000'],
 'high': ['27858.00000000',
  '27858.00000000',
  '27858.00000000',
  '27858.00000000',
  '27858.00000000'],
 'low': ['24050.00000000',
  '24050.00000000',
  '24050.00000000',
  '24050.00000000',
  '24050.00000000'],
 'close': ['27021.98000000',
  '27021.98000000',
  '27021.96000000',
  '27021.96000000',
  '27021.95000000']}